# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import gc
from pathlib2 import Path
import numpy as np
import pandas as pd

sys.path.append('..')
from src import config

display_settings = {
    'max_columns': 999,
    'expand_frame_repr': True,
    'max_rows': 999,
    'precision': 4,
    'show_dimensions': True
}

for op, value in display_settings.items():
    pd.set_option("display.{}".format(op), value)

%matplotlib inline

# Data

## Load datasets

In [3]:
# prices
dtypes_prices = {
    'store_id': 'str',
    'item_id': 'str',
    'wm_yr_wk': 'int16',
    'selle_price': 'float32'
}
cat_cols_prices = ['store_id', 'item_id']

df_prices = pd.read_csv(config.DATA_FILES['prices'], dtype=dtypes_prices)
df_prices[cat_cols_prices] = df_prices[cat_cols_prices].astype('category')

print(f'Prices series dataset:{df_prices.shape}')
df_prices.head()

Prices series dataset:(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [4]:
# calendar
dtypes_calendar = {
    'date': 'str',
    'wm_yr_wk': 'int16',
    'weekday': 'str',
    'wday': 'int8',
    'month': 'int8',
    'year': 'int16',
    'd': 'str',
    'event_name_1': 'str',
    'event_type_1': 'str',
    'event_name_2': 'str',
    'event_type_2': 'str',
    'snap_CA': 'bool',
    'snap_TX': 'bool',
    'snap_WI': 'bool',
}
cat_cols_calendar = ['weekday', 'event_type_1', 'event_type_2']

df_calendar = pd.read_csv(config.DATA_FILES['calendar'], dtype=dtypes_calendar, parse_dates=['date'])
df_calendar[cat_cols_calendar] = df_calendar[cat_cols_calendar].astype('category')

print(f'Calendar dataset:{df_calendar.shape}')
df_calendar.head()

Calendar dataset:(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,False,False,False
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,False,False,False
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,False,False,False
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,True,True,False
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,True,False,True


In [5]:
#df_calendar_test = df_calendar.copy()

#for col in ['event_type_1', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']:
#    df_calendar_test[f'{col}_days_since'] = df_calendar_test.apply(lambda x: np.argmax(df_calendar_test.iloc[x.name::-1][col].tolist()), axis=1)

#df_calendar_test

In [6]:
horizon = 28
last_day = 1913

dtypes_sales = {
    'id': 'str',
    'item_id': 'str',
    'dept_id': 'str',
    'cat_id': 'str',
    'store_id': 'str',
    'state_id': 'str',
}
dtypes_sales_ts = {f'd_{day}': 'int16' for day in range(1, last_day)}
cat_cols_sales = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

df_sales = pd.read_csv(config.DATA_FILES['sales'], dtype=dict(dtypes_sales, **dtypes_sales_ts))
df_sales[cat_cols_sales] = df_sales[cat_cols_sales].astype('category')

# append days to forecast
for i in range (horizon):
    df_sales[f'd_{last_day + i + 1}'] = np.nan

print(f'Sales series dataset:{df_sales.shape}')
df_sales.head()

Sales series dataset:(30490, 1947)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365,d_366,d_367,d_368,d_369,d_370,d_371,d_372,d_373,d_374,d_375,d_376,d_377,d_378,d_379,d_380,d_381,d_382,d_383,d_384,d_385,d_386,d_387,d_388,d_389,d_390,d_391,d_392,d_393,d_394,d_395,d_396,d_397,d_398,d_399,d_400,d_401,d_402,d_403,d_404,d_405,d_406,d_407,d_408,d_409,d_410,d_411,d_412,d_413,d_414,d_415,d_416,d_417,d_418,d_419,d_420,d_421,d_422,d_423,d_424,d_425,d_426,d_427,d_428,d_429,d_430,d_431,d_432,d_433,d_434,d_435,d_436,d_437,d_438,d_439,d_440,d_441,d_442,d_443,d_444,d_445,d_446,d_447,d_448,d_449,d_450,d_451,d_452,d_453,d_454,d_455,d_456,d_457,d_458,d_459,d_460,d_461,d_462,d_463,d_464,d_465,d_466,d_467,d_468,d_469,d_470,d_471,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,...,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_

## Melt and merge

In [7]:
id_vars = ['id'] + cat_cols_sales
df_sales = df_sales.melt(id_vars=id_vars, var_name='d', value_name='sold_qty')

df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold_qty
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


In [8]:
df_sales = df_sales.merge(df_calendar, on='d', copy=False)
del df_calendar
gc.collect()

df_sales = df_sales.merge(df_prices, on=['store_id', 'item_id', 'wm_yr_wk'], copy=False)
del df_prices
gc.collect()

df_sales.d = df_sales.d.str.replace('d_', '').astype('int16')
df_sales.drop(['wm_yr_wk', 'weekday'], axis=1, inplace=True)
df_sales.rename({'d': 'day_ind'}, axis=1, inplace=True)

print(f'Consolidated sales series dataset:{df_sales.shape}')
df_sales.head()

Consolidated sales series dataset:(46881677, 20)


,id,item_id,dept_id,cat_id,store_id,state_id,day_ind,sold_qty,date,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,False,False,False,0.46
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,2,15.0,2011-01-30,2,1,2011,NaN,NaN,NaN,NaN,False,False,False,0.46
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,3,0.0,2011-01-31,3,1,2011,NaN,NaN,NaN,NaN,False,False,False,0.46
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,4,0.0,2011-02-01,4,2,2011,NaN,NaN,NaN,NaN,True,True,False,0.46
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,5,0.0,2011-02-02,5,2,2011,NaN,NaN,NaN,NaN,True,False,True,0.46


## Save to parquet

In [9]:
path = config.get_processed_filename('data_merged.parquet')
df_sales.to_parquet(path)

del df_sales
gc.collect()

print(f'Consolidated dataset size:{(Path(path).stat().st_size / 1e6):,.2f}Mo')

Consolidated dataset size:354.31Mo
